In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/cleaned_news.csv")

In [2]:
# ! pip install transformers sentence-transformers vllm torch rank_bm25 faiss-cpu accelerate

In [3]:
from transformers import AutoTokenizer
from vllm import LLM
from temporal_rag import (
    TemporalRAGIndexBuilder,
    TemporalRAGPipeline,
    tokenize_ru,
)

# строим индексы для ретривала
builder = TemporalRAGIndexBuilder(
    save_dir="indexes_test",
    encoder_name="intfloat/multilingual-e5-large",
    device="cuda",
)

art = builder.build(df, batch_size=64)
builder.save(art)

[2026-01-15 01:26:35] INFO loader.py:120: Loading faiss with AVX512 support.
[2026-01-15 01:26:35] INFO loader.py:122: Successfully loaded faiss with AVX512 support.
[2026-01-15 01:26:43] INFO SentenceTransformer.py:227: Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/1927 [00:00<?, ?it/s]

E_docs: <class 'numpy.ndarray'> float32 (123273, 1024) is ndarray: True


In [4]:
# from transformers import AutoTokenizer
# from vllm import LLM
# from temporal_rag import (
#     TemporalRAGIndexBuilder,
#     TemporalRAGPipeline,
#     tokenize_ru,
# )

# # выгружаем индексы для ретривала
# builder = TemporalRAGIndexBuilder(
#     save_dir="indexes_test",
#     encoder_name="intfloat/multilingual-e5-large",
#     device="cuda",
# )

# df2, encoder, index, bm25 = builder.load(df, build_encoder=True)

In [6]:
# подгружаем ллм
from transformers import AutoTokenizer
from vllm import LLM

MODEL = "Qwen/Qwen2.5-32B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=False)

llm = LLM(
    model=MODEL,
    dtype="bfloat16",
    max_model_len=16000,
    gpu_memory_utilization=0.89,
    disable_log_stats=True,
    max_num_seqs=32
)

INFO 01-15 01:45:13 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'max_model_len': 16000, 'gpu_memory_utilization': 0.89, 'max_num_seqs': 32, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-32B-Instruct'}
INFO 01-15 01:45:14 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-15 01:45:14 [model.py:1661] Using max model len 16000
INFO 01-15 01:45:14 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=1343) INFO 01-15 01:45:21 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='Qwen/Qwen2.5-32B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, k

(EngineCore_DP0 pid=1343) /home/mlcore/conda/lib/python3.10/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=1343) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=1343)   warnings.warn(


(EngineCore_DP0 pid=1343) INFO 01-15 01:45:25 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:09,  1.61it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:11,  1.30it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:02<00:12,  1.16it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:03<00:11,  1.15it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:04<00:10,  1.16it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:05<00:09,  1.16it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:05<00:08,  1.16it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:06<00:07,  1.15it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:07<00:06,  1.15it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:08<00:05,  1.22it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=1343) INFO 01-15 01:45:41 [default_loader.py:308] Loading weights took 14.79 seconds
(EngineCore_DP0 pid=1343) INFO 01-15 01:45:42 [gpu_model_runner.py:3659] Model loading took 61.0375 GiB memory and 18.923066 seconds
(EngineCore_DP0 pid=1343) INFO 01-15 01:45:53 [backends.py:643] Using cache directory: /home/mlcore/.cache/vllm/torch_compile_cache/956bf58fae/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=1343) INFO 01-15 01:45:53 [backends.py:703] Dynamo bytecode transform time: 11.64 s
(EngineCore_DP0 pid=1343) INFO 01-15 01:46:00 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=1343) INFO 01-15 01:46:17 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 16.79 s
(EngineCore_DP0 pid=1343) INFO 01-15 01:46:17 [monitor.py:34] torch.compile takes 28.43 s in total
(EngineCore_DP0 pid=1343) INFO 01-15 01:46:18 [gpu_worker.py:375] Available KV cache memory: 8.01 GiB
(EngineCore_DP0 pid=1343) 

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 11/11 [00:01<00:00, 10.51it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 7/7 [00:00<00:00, 11.08it/s]


(EngineCore_DP0 pid=1343) INFO 01-15 01:46:21 [gpu_model_runner.py:4587] Graph capturing finished in 2 secs, took 0.92 GiB
(EngineCore_DP0 pid=1343) INFO 01-15 01:46:21 [core.py:259] init engine (profile, create kv cache, warmup model) took 39.32 seconds
INFO 01-15 01:46:23 [llm.py:360] Supported tasks: ['generate']


In [10]:
# формируем пайплайн
from system_prompt import SYSTEM_PROMPT

pipe = TemporalRAGPipeline(
    df=art.df,
    index=art.index,
    encoder=art.encoder,
    bm25=art.bm25,
    tokenize_fn=tokenize_ru,
    sum_llm=llm,
    sum_tokenizer=tokenizer,
    judge_llm=llm,
    judge_tokenizer=tokenizer,
    system_prompt=SYSTEM_PROMPT,  
)

In [11]:
# ответ
out = pipe.answer(
    query="Почему кофе дорожает?",
    anchor_date="2025-09-04", # дата обзора
    k_retrieve=150, # глубина ретривала
    topN_each=2000, # количество документов для каждого метода ретривала (бм25 / семантика)
    k_docs=30, # количество документов в суммаризации
    snip_chars=1000, # размер каждой новости
    max_new_tokens=5000, # ограничение ответа
    max_window_days=365, # максимальный промежуток времени ретривала
    w_time=0.5, # вес даты для ретривала
    hot_window_days=30, # окно для определения актуальной новости
    hot_ratio=0.7, # доля актуальных новостей в ответе из топ-к
)

print(out.summary)

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

...

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

### 1) Запрос и актуальная дата
* Запрос: Почему кофе дорожает?
* Актуальная дата: 2025-09-04

### 2) Дайджест
Последнее время, с 2025-08-30, кофе сорта робуста и арабика подорожал на мировых рынках, увеличившись в цене в 1,5 раза и на 29% соответственно. Это связано с проблемами с урожаем в Бразилии и Вьетнаме из-за неблагоприятных погодных условий, таких как засуха и проливные дожди. В рамках проекта по борьбе с производством кокаина в Колумбии, фермеров убеждают выращивать кофе вместо коки, что также может повлиять на рынок кофе. Ранее, с 2025-08-01, в России ожидалось новое подорожание кофе на 20-40% из-за проблем с урожаем в Бразилии и Вьетнаме. В целом, тема подорожания кофе остается актуальной и вирусной, поскольку цены продолжают расти из-за неблагоприятных погодных условий и проблем с урожаем.

### 3) Таймлайн
* 2025-09-01 — Фермеров в Колумбии убеждают выращивать кофе вместо коки
* 2025-08-30 — Кофе сорта робуста и арабика подорожал на мировых рынках
* 2025-08-26 — Шоколад дл